In [1]:
import pandas
import reactome2py
from reactome2py import analysis, content, utils
from collections import defaultdict
from itertools import chain
from operator import methodcaller

# https://www.ncbi.nlm.nih.gov/pubmed/31502168 
path = "data/marker_matrix_gene.csv"

df = pandas.read_csv(path, index_col=0)
df = df.apply(lambda x : x.str.split(","), axis=0) # 1 to many relation of protiens to genes - relapce cells string to list type
row_dict = df.T.to_dict('records')

def imggenes2dict(row_dict):
    dd = defaultdict(list)
    dict_items = map(methodcaller('items'), row_dict)
    
    for k, v in chain.from_iterable(dict_items):
        dd[k].extend(v)
    
    img_dict = dict(dd)
    return(img_dict)

img_dict = imggenes2dict(row_dict)
result = [analysis.identifiers(ids=",".join(d)) for d in img_dict.values()]
tokens = [r['summary']['token'] for r in result]


def get_pathways(t):
    token_result = analysis.token(token=t, species='Homo sapiens', page_size='-1', page='-1', sort_by='ENTITIES_FDR', 
                                  order='ASC', resource='TOTAL', p_value='1', include_disease=True, 
                                  min_entities=None, max_entities=None)
    l = [p['name'] for p in token_result['pathways']]
    l.sort() 
    return(l)

pathways = [get_pathways(t) for t in tokens]